In [1]:
import numpy as np
import io_mesh as io
import subprocess
import argparse
import os
import copy
import matplotlib.pyplot as plt
import subprocess
import glob
%matplotlib inline

/home/users/kwagstyl/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/users/kwagstyl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

# TODO replace with function from io_mesh?
def load_surface(surfname):
    """Loads obj surface.
    Returns:
        n_vert (int, number of vertices)
        coords, 
        normals, 
        triangles
    """
    polys=[]
    p=0
    k=0
    with open(surfname, 'r') as fp:
        for i, line in enumerate(fp):
            if i==0:
                # Number of vertices
                line1 = line.split()
                n_vert = int(line1[6])
                coords = np.zeros([ n_vert, 3])
                norms = np.zeros([ n_vert, 3])
            elif 0 < i < n_vert+1:
                # get coordinates
                coords[ i-1 ] = list(map( float, line.split() ))
            elif n_vert+1 < i < 2*n_vert + 2:
                # get normals (we just write them back out, not recomputing)
                norms[p]=line.split()
                p+=1
            elif i==2*n_vert+3:
                # get number of polygons. Not needed any more
                n_poly=int(line)
            elif i==2*n_vert+4:
                # get colour info
                col=line
            elif i>2*n_vert+5 and k==0:
                if not line.strip():
                    k=1
            elif k==1:
                # get polygons
                polys.extend(line.split())
    polys = list(map(int,polys))
    tris = list(chunks(polys,3))
    return n_vert, coords, norms, tris;

def load_intensities(intensity_file):
    with open(intensity_file,'r') as f:
            for n in range(4): 
                f.readline()
            lst=f.readline().split('\t')[1:-1]
            vector=[]
            for i in lst:
                try:
                    vector.append(float(i))
                except ValueError:
                    vector.append(0)          
    return vector 

In [3]:
surf_dir='/data1/users/kwagstyl/bigbrain/VAST/surface_data/obj_surfaces/'
profile_dir='/data1/users/kwagstyl/bigbrain/VAST/surface_data/profile_data/'
bin_dir='/data1/users/kwagstyl/quarantines/Linux-x86_64/bin/'
vol_dir='/data1/users/kwagstyl/bigbrain/volumes/'
os.makedirs(surf_dir, exist_ok=True)
os.makedirs(profile_dir, exist_ok=True)


In [4]:
#calculate blocks to sample from legend1000.mnc, a volume containing block indices
hemis=['right','left']
for hemi in hemis:
    subprocess.call('{} -nearest_neighbour {} {} {}'.format(os.path.join(bin_dir,'volume_object_evaluate'),
                                                    os.path.join(vol_dir,'legend1000.mnc'),
                                                    os.path.join(surf_dir,'gray_{}.obj'.format(hemi)),
                                                    os.path.join(profile_dir,'block_to_sample_{}.txt'.format(hemi))),
    shell=True)

In [ ]:
#generate equidistant profiles
smoothings=['geo','raw']
n_surfs=200
for hemi in hemis:
    print('generating equidistant coordinates')
    n_vert, white_coords, d, d =load_surface(os.path.join(surf_dir,'white_{}.obj'.format(hemi)))
    n_vert, gray_coords, d, d =load_surface(os.path.join(surf_dir,'gray_{}.obj'.format(hemi)))
    vector=(white_coords-gray_coords)/(n_surfs-1)
    hemi_coords=np.zeros((n_surfs,n_vert,3))
    hemi_intensities=np.zeros((n_surfs,n_vert))
    for surf_index in range(n_surfs):
        hemi_coords[surf_index]=gray_coords+vector*surf_index
        
        
    
    blocks=np.loadtxt(os.path.join(profile_dir,'block_to_sample_{}.txt'.format(hemi))).astype(int)
    hemi_blocks=np.unique(blocks)
    for smoothing in smoothings:
        print('sampling {} blocks'.format(smoothing))
        for block in hemi_blocks:
            print('sampling block:{}'.format(block))
            block_coordinates=hemi_coords[:,blocks==block,:]
            verts_in_block=block_coordinates.shape[1]
        #reshape into streamlines
            block_hemi_coords_streamlines=np.reshape(block_coordinates,(n_surfs*verts_in_block,3),order='F')
        #write out streamlines into txt file
            np.savetxt(os.path.join(profile_dir,'coordinates_block.txt'),block_hemi_coords_streamlines)
        #sample streamlines from block (dummy with full low res first)
            if smoothing == 'geo':
                glimfile = ' /data1/users/kwagstyl/bigbrain/blocks20/block20-{0:0>4}_geo.mnc\n'.format(block)
                txtname='_geo'
            else:
                glimfile = ' /data1/users/kwagstyl/bigbrain/blocks20/block20-{0:0>4}.mnc\n'.format(block)
                txtname=''
            #glimfile=' /data1/users/kwagstyl/bigbrain/volumes/full8_300um.mnc\n'
            dumglim = ' /data1/users/kwagstyl/bigbrain/volumes/full8_1000um.mnc\n'
            with open(os.path.join(profile_dir,'glimblock'),'w') as f:
                f.write(dumglim+glimfile)
            subprocess.call('{} {} {} {} >/dev/null'.format(os.path.join(bin_dir,'print_world_values'),
                                                                                            os.path.join(profile_dir,'glimblock'),
                                                                                            os.path.join(profile_dir,'coordinates_block.txt'),
                                                                                            os.path.join(profile_dir,'block_values_{0:0>4}{1}.txt'.format(block,txtname))
                                                                                            ),shell=True)
              #import profiles for block, put into overall profiles matrix
            block_intensities=load_intensities(os.path.join(profile_dir,'block_values_{0:0>4}{1}.txt'.format(block,txtname)))
            hemi_intensities[:,blocks[:n_vert]==block]=np.reshape(block_intensities,(n_surfs,verts_in_block),order='F')
        #save out overall profiles matrix
        np.savetxt(os.path.join(profile_dir,'euclid_{}_profiles_{}.txt'.format(smoothing,hemi)),hemi_intensities.T,fmt='%.2f')

generating equidistant coordinates


In [ ]:
#generate coordinates, TODO
for hemi in hemis:
    smoothing=''
    print('generating coordinates')
    n_vert=163842
    n_surfs=200
    hemi_coords=np.zeros((n_surfs,n_vert,3))
    hemi_intensities=np.zeros((n_surfs,n_vert))
    surf_refs=np.linspace(0,1,n_surfs)
    surf_names=sorted(glob.glob('equivolume_surfaces/equi_volume_'+hemi+'*'))
    for surf_index in range(n_surfs):
        surfname=surf_names[surf_index]
        hemi_coords[surf_index]=load_coords(surfname,n_vert)
#generate block coordinates
    print('sampling blocks')
    blocks=np.loadtxt('block_to_sample_'+hemi+'.txt').astype(int)
    hemi_blocks=np.unique(blocks[:n_vert])
    for block in hemi_blocks:
        print(block)
        block_coordinates=hemi_coords[:,blocks[:n_vert]==block,:]
        verts_in_block=block_coordinates.shape[1]
    #reshape into streamlines
        block_hemi_coords_streamlines=np.reshape(block_coordinates,(n_surfs*verts_in_block,3),order='F')
    #write out streamlines into txt file
        np.savetxt('coordinates_block.txt',block_hemi_coords_streamlines)
    #sample streamlines from block (dummy with full low res first)
        if smoothing == 'geo':
            glimfile = ' /data1/users/kwagstyl/bigbrain/blocks20/block20-{0:0>4}_geo.mnc\n'.format(block)
            txtname='_geo'
        else:
            glimfile = ' /data1/users/kwagstyl/bigbrain/blocks20/block20-{0:0>4}.mnc\n'.format(block)
            txtname=''
        #glimfile=' /data1/users/kwagstyl/bigbrain/volumes/full8_300um.mnc\n'
        dumglim = ' /data1/users/kwagstyl/bigbrain/volumes/full8_1000um.mnc\n'
        with open('glimblock','w') as f:
            f.write(dumglim+glimfile)
        subprocess.call('/data1/users/kwagstyl/quarantines/Linux-x86_64/bin/print_world_values glimblock coordinates_block.txt block_values_{0:0>4}{1}.txt >/dev/null'.format(block,txtname),shell=True)
        #import profiles for block, put into overall profiles matrix
        block_intensities=load_intensities('block_values_{0:0>4}{1}.txt'.format(block,txtname))
        hemi_intensities[:,blocks[:n_vert]==block]=np.reshape(block_intensities,(n_surfs,verts_in_block),order='F')
    #save out overall profiles matrix
    np.savetxt(smoothing+'_profiles_'+hemi+'.txt',hemi_intensities.T)


generating coordinates
sampling blocks
53
54
55
56
57
58
